In [2]:
# 🔧 Configuración e imports
import os
import time
import warnings
from pathlib import Path
from typing import List, Dict, Any, TypedDict

from dotenv import load_dotenv
warnings.filterwarnings("ignore")
load_dotenv()

# Credenciales requeridas
required_keys = ["OPENAI_API_KEY", "QDRANT_URL", "QDRANT_API_KEY"]
missing = [k for k in required_keys if not os.getenv(k)]
if missing:
    print(f"⚠️ Faltan variables de entorno: {missing}")
else:
    print("✅ Credenciales OK")

# Rutas
PDF_PATH = "/Users/pablolastrabachmann/DiploGenAI/rag_langgraph/guia_carretera_1.pdf"
TEXT_PATH = "/Users/pablolastrabachmann/DiploGenAI/rag_langgraph/info_carretera.txt"
COLLECTION_NAME = "carretera_austral"


✅ Credenciales OK


In [2]:
# 📄 Carga del PDF con fallback a OCR
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Múltiples loaders de PDF y fallback a OCR

def load_pdf_with_fallback(pdf_path: str) -> List[Document]:
    pages: List[Document] = []
    '''
   # 1) MarkItDown (markdown extractor con soporte multi-formato)
    try:
        from markitdown import MarkItDown
        md = MarkItDown()
        md_result = md.convert(pdf_path)
        txt = (getattr(md_result, "text_content", None) or "").strip()
        if txt:
            print(f"✅ MarkItDown: {len(txt)} chars")
            return [Document(page_content=txt, metadata={"page": "all", "source": pdf_path, "method": "MarkItDown"})]
        print("⚠️ MarkItDown: sin texto")
    except Exception as e:
        print(f"⚠️ MarkItDown falló: {e}")
    '''
    # 2) PyMuPDF (fitz)
    try:
        import fitz
        doc = fitz.open(pdf_path)
        pages = []
        for i in range(len(doc)):
            page = doc.load_page(i)
            txt = page.get_text()
            if txt.strip():
                pages.append(Document(page_content=txt, metadata={"page": i+1, "source": pdf_path}))
        doc.close()
        if pages:
            print(f"✅ PyMuPDF: {len(pages)} páginas")
            return pages
        print("⚠️ PyMuPDF: sin texto")
    except Exception as e:
        print(f"⚠️ PyMuPDF falló: {e}")

    # 3) PDFPlumber
    try:
        import pdfplumber
        pages = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, p in enumerate(pdf.pages):
                txt = p.extract_text() or ""
                if txt.strip():
                    pages.append(Document(page_content=txt, metadata={"page": i+1, "source": pdf_path}))
        if pages:
            print(f"✅ PDFPlumber: {len(pages)} páginas")
            return pages
        print("⚠️ PDFPlumber: sin texto")
    except Exception as e:
        print(f"⚠️ PDFPlumber falló: {e}")

    # 4) PyPDFLoader
    try:
        from langchain_community.document_loaders import PyPDFLoader
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        if pages and len(pages[0].page_content.strip()) > 50:
            print(f"✅ PyPDFLoader: {len(pages)} páginas")
            return pages
        print("⚠️ PyPDFLoader: contenido insuficiente")
    except Exception as e:
        print(f"⚠️ PyPDFLoader falló: {e}")

    # 5) UnstructuredPDFLoader (opcional)
    try:
        from langchain_community.document_loaders import UnstructuredPDFLoader
        loader = UnstructuredPDFLoader(pdf_path)
        pages = loader.load()
        if pages:
            print(f"✅ UnstructuredPDFLoader: {len(pages)} documentos")
            return pages
    except Exception as e:
        print(f"⚠️ UnstructuredPDFLoader falló: {e}")

    # 6) pypdf básico
    try:
        import pypdf
        pages = []
        with open(pdf_path, 'rb') as f:
            pdf = pypdf.PdfReader(f)
            for i, pg in enumerate(pdf.pages):
                txt = pg.extract_text() or ""
                if txt.strip():
                    pages.append(Document(page_content=txt, metadata={"page": i+1, "source": pdf_path}))
        if pages:
            print(f"✅ pypdf: {len(pages)} páginas")
            return pages
        print("⚠️ pypdf: sin texto")
    except Exception as e:
        print(f"⚠️ pypdf falló: {e}")

    # 7) OCR con EasyOCR
    print("🔄 Intentando OCR (EasyOCR)...")
    try:
        import fitz
        import easyocr
        doc = fitz.open(pdf_path)
        reader = easyocr.Reader(['es', 'en'])
        pages = []
        for i in range(len(doc)):
            page = doc.load_page(i)
            pix = page.get_pixmap()
            img = pix.tobytes("png")
            results = reader.readtext(img)
            text = " ".join([t for (_, t, conf) in results if conf > 0.5])
            if text.strip():
                pages.append(Document(page_content=text, metadata={"page": i+1, "source": pdf_path, "method": "OCR"}))
        doc.close()
        if pages:
            print(f"✅ OCR: {len(pages)} páginas")
            return pages
    except Exception as e:
        print(f"❌ OCR falló: {e}")

    raise RuntimeError("No se pudo extraer texto del PDF")

raw_documents = []
if Path(PDF_PATH).exists():
    try:
        raw_documents = load_pdf_with_fallback(PDF_PATH)
        print(f"📚 Documentos extraídos: {len(raw_documents)}")
    except Exception as e:
        print(f"❌ Error cargando PDF: {e}")
else:
    print(f"❌ PDF no encontrado en {PDF_PATH}")


✅ PyMuPDF: 54 páginas
📚 Documentos extraídos: 54


In [4]:
# 📄 Carga del TXT turístico
from langchain.schema import Document

if Path(TEXT_PATH).exists():
    try:
        with open(TEXT_PATH, "r", encoding="utf-8") as f:
            txt_content = f.read().strip()
        if txt_content:
            raw_documents.append(
                Document(
                    page_content=txt_content,
                    metadata={
                        "page": "all",
                        "source": TEXT_PATH,
                        "title": "Información turística Carretera Austral",
                        "doc_type": "turismo_texto"
                    },
                )
            )
            print(f"✅ TXT cargado: {len(txt_content)} chars")
        else:
            print("⚠️ TXT sin contenido")
    except Exception as e:
        print(f"❌ Error leyendo TXT: {e}")
else:
    print(f"❌ TXT no encontrado en {TEXT_PATH}")

print(f"📚 Total documentos crudos: {len(raw_documents)}")

✅ TXT cargado: 25193 chars
📚 Total documentos crudos: 55


In [ ]:
# 🗃️ Conexión a Qdrant y creación/uso de colección
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_experimental.text_splitter import SemanticChunker

# Embeddings & LLM
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)
llm = ChatOpenAI(model="gpt-5-2025-08-07")
llm_small = ChatOpenAI(model="gpt-5-mini-2025-08-07")

# Qdrant
qdrant = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))
print("✅ Qdrant OK")

# Preprocesado: segmentación por bloques y protección de abreviaturas
import re
from langchain.schema import Document

ABBREVIATION_UNITS_PATTERN = re.compile(r"(?i)(\b\d+\s*(?:km|kms|km/h|h|hr|hrs|min|mins|m|cm|mm))\.")
ABBREVIATIONS = [
    "aprox.", "etc.", "p. ej.", "p.ej.", "S.E.", "N.E.", "N.O.", "S.O.",
    "Sr.", "Sra.", "Sres.", "Dr.", "Dra.", "Ud.", "Uds.", "Av.", "No.", "Nº.",
]
ABBR_PATTERN = re.compile(r"(" + "|".join(re.escape(a) for a in ABBREVIATIONS) + ")", re.IGNORECASE)
PLACEHOLDER_DOT = "∯"


def protect_abbreviations(text: str) -> str:
    # Unidades con punto (e.g., "423 km.")
    text = ABBREVIATION_UNITS_PATTERN.sub(lambda m: m.group(1) + PLACEHOLDER_DOT, text)
    # Abreviaturas comunes
    text = ABBR_PATTERN.sub(lambda m: m.group(0).replace(".", PLACEHOLDER_DOT), text)
    return text


def unprotect_abbreviations(text: str) -> str:
    return text.replace(PLACEHOLDER_DOT, ".")


def blockify_documents(documents):
    new_docs = []
    for doc in documents:
        # Separar por bloques/ párrafos (2+ saltos de línea)
        blocks = re.split(r"\n{2,}", doc.page_content)
        for bi, block in enumerate(blocks):
            block_text = block.strip()
            if len(block_text) < 60:
                continue
            meta = dict(doc.metadata)
            meta["block_id"] = bi
            # Proteger abreviaturas antes del chunking semántico
            new_docs.append(Document(page_content=protect_abbreviations(block_text), metadata=meta))
    return new_docs

input_documents = blockify_documents(raw_documents) if 'raw_documents' in locals() and raw_documents else []

# Permitir recrear colección para reindexar con mejoras de chunking
RECREATE_COLLECTION = False

# Vector store (crear si no existe o si forzamos recreación)
existing = [c.name for c in qdrant.get_collections().collections]
if COLLECTION_NAME in existing and not RECREATE_COLLECTION:
    print(f"🔗 Conectando a colección existente '{COLLECTION_NAME}'...")
    vector_store = QdrantVectorStore(client=qdrant, collection_name=COLLECTION_NAME, embedding=embeddings)
else:
    if COLLECTION_NAME in existing and RECREATE_COLLECTION:
        print(f"♻️ Reindexando: eliminando colección existente '{COLLECTION_NAME}'...")
        qdrant.delete_collection(collection_name=COLLECTION_NAME)
    if raw_documents:
        print(f"🆕 Creando colección '{COLLECTION_NAME}'...")
        # Semantic chunking basado en embeddings
        semantic_chunker = SemanticChunker(
            embeddings,
            buffer_size=2,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95,
            sentence_split_regex=r"(?<=(?<![Kk][Mm])[.?!])\s+",
            min_chunk_size=180,
        )
        # Usar documentos preprocesados por bloques/abreviaturas
        base_docs = input_documents if input_documents else raw_documents
        chunks = semantic_chunker.split_documents(base_docs)
        # Desproteger abreviaturas en los chunks
        for i, ch in enumerate(chunks):
            ch.page_content = unprotect_abbreviations(ch.page_content)
            ch.metadata.update({
                "source_title": "Guía Carretera Austral",
                "collection": COLLECTION_NAME,
                "chunk_id": i,
                "categoria": "turismo_carretera_austral",
                "char_count": len(ch.page_content)
            })
        vector_store = QdrantVectorStore.from_documents(
            documents=chunks,
            embedding=embeddings,
            url=os.getenv("QDRANT_URL"),
            api_key=os.getenv("QDRANT_API_KEY"),
            collection_name=COLLECTION_NAME,
            force_recreate=True,
        )
        print(f"✅ Colección creada con {len(chunks)} chunks")
    else:
        raise RuntimeError("No hay documentos para crear la colección y la colección no existe")

# Retriever: vectorial (Qdrant) + utilidades léxicas on-demand (sin cargar 2000 docs)
from langchain_community.retrievers import BM25Retriever

# Normalizar guiones de corte de línea (e.g., "Lagu-\nna" -> "Laguna")
import re as _re

def normalize_hyphens(text: str) -> str:
    return _re.sub(r"(\w)-\n(\w)", r"\1\2", text)

import unicodedata
import string

def simple_preprocess(text: str):
    # minúsculas, quitar acentos, limpiar puntuación ligera
    text = text.lower()
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    text = text.replace('\n', ' ')
    table = str.maketrans({ch: ' ' for ch in '“”"' + string.punctuation})
    text = text.translate(table)
    tokens = [t for t in text.split() if len(t) > 1]
    return tokens

# Vector retriever (no BM25 global)
vec_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 40, "lambda_mult": 0.5},
)
print("✅ Vector retriever listo (BM25 solo on-demand)")


✅ Qdrant OK
🔗 Conectando a colección existente 'carretera_austral'...
✅ Vector retriever listo (BM25 solo on-demand)


In [4]:
# 🕸️ Grafo Agentic RAG (LangGraph)
from langgraph.graph import StateGraph, END
from langchain.callbacks import get_openai_callback

class TourismRAGState(TypedDict):
    query: str
    original_query: str
    documents: List[Document]
    sources: List[str]
    doc_quality_score: float
    retrieval_attempts: int
    should_rewrite: bool
    answer: str
    workflow_steps: List[str]
    total_tokens: int

# Utilidades
from numpy import dot
from numpy.linalg import norm

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(doc.page_content for doc in docs)

def extract_sources(docs: List[Document]) -> List[str]:
    out = []
    for d in docs:
        title = d.metadata.get("title") or d.metadata.get("source_title") or f"Página {d.metadata.get('page','?')}"
        out.append(title)
    return out

def calculate_doc_relevance(docs: List[Document], query: str) -> float:
    if not docs:
        return 0.0
    try:
        q = embeddings.embed_query(query)
        doc_vecs = embeddings.embed_documents([d.page_content for d in docs])
        sims = [max(0, float(dot(q, v) / (norm(q) * norm(v)))) for v in doc_vecs]
        return sum(sims) / len(sims)
    except Exception:
        return 0.0

# Nodos
from typing import Any

def retrieve_node(state: TourismRAGState) -> Dict[str, Any]:
    query = state["query"]
    print(f"  🔍 Recuperando documentos para: {query}")

    # 1) Candidatos vectoriales desde Qdrant (sin grandes cargas en memoria)
    vec_docs = vec_retriever.invoke(query)

    # 2) Normaliza texto para el scorer léxico
    for d in vec_docs:
        d.page_content = normalize_hyphens(d.page_content)

    # 3) BM25 on-demand SOLO sobre estos candidatos (sin corpus global)
    bm25_local = BM25Retriever.from_documents(vec_docs, k=6, preprocess_func=simple_preprocess)
    bm25_docs = bm25_local.invoke(query)

    # 4) Fusión simple preservando orden (lexical primero, luego vectorial) y sin duplicados
    def key(d: Document):
        return (
            d.metadata.get("_id"),
            d.metadata.get("source"),
            d.metadata.get("page"),
            d.metadata.get("chunk_id"),
        )

    seen = set()
    merged: List[Document] = []
    for d in bm25_docs + vec_docs:
        k = key(d)
        if k in seen:
            continue
        seen.add(k)
        merged.append(d)

    # 5) Limita resultados finales
    final_k = 8
    docs = merged[:final_k]

    score = calculate_doc_relevance(docs, query)
    sources = extract_sources(docs)
    steps = state.get("workflow_steps", [])
    steps.append(f"retrieve (2-stage): vec={len(vec_docs)} bm25={len(bm25_docs)} final={len(docs)} quality={score:.3f}")
    return {
        "documents": docs,
        "doc_quality_score": score,
        "sources": sources,
        "workflow_steps": steps,
        "retrieval_attempts": state.get("retrieval_attempts", 0) + 1,
    }

def grade_node(state: TourismRAGState) -> Dict[str, Any]:
    score = state["doc_quality_score"]
    should_rewrite = score < 0.5
    steps = state["workflow_steps"]
    steps.append(f"grade: score={score:.3f}, rewrite={should_rewrite}")
    return {"should_rewrite": should_rewrite, "workflow_steps": steps}

def rewrite_node(state: TourismRAGState) -> Dict[str, Any]:
    prompt = f"""
    Actúa como analista de consultas para un sistema RAG turístico sobre la Carretera Austral (Ruta 7, Patagonia, Chile).

    Objetivo: reescribir la consulta para maximizar el recall en la búsqueda semántica, preservando la intención original y el idioma.

    Instrucciones:
    - No inventes datos nuevos ni cambies la intención.
    - Explicita, cuando existan, estos elementos: tramo u origen–destino, sentido (N→S o S→N), cantidad de días/fechas/temporada, medio de transporte (auto/4x4, moto, bici, a pie, bus, ferry/barcaza), tipo de actividades (miradores, trekking, navegación, parques), restricciones (clima, niños, presupuesto, tiempo), y logística (ferries/barcazas, horarios, accesos, combustible).
    - Añade sinónimos del dominio SOLO si aumentan el recall: "Carretera Austral"/"Ruta 7"; "ferry"/"barcaza"/"transbordador"; "sendero"/"trekking"; "acceso"/"entrada"; "horario"/"itinerario"; "Parque Nacional"/"CONAF".
    - Evita vaguedad ("mejor", "lindo"); usa criterios comparables si están implícitos (distancia, tiempo, estado del camino, necesidad de 4x4).
    - Formato de salida: devuelve SOLO la consulta reescrita en UNA ÚNICA línea, sin comillas ni explicación adicional.

    Consulta original: {state['original_query']}
    Consulta actual: {state['query']}

    Consulta reescrita:
    """
    try:
        resp = llm_small.invoke(prompt)
        new_q = resp.content.strip()
        steps = state["workflow_steps"]
        steps.append(f"rewrite: '{state['query']}' -> '{new_q}'")
        return {"query": new_q, "workflow_steps": steps}
    except Exception as e:
        print(f"❌ Rewrite error: {e}")
        return {"workflow_steps": state["workflow_steps"]}

def generate_node(state: TourismRAGState) -> Dict[str, Any]:
    docs = state["documents"]
    context = format_docs(docs)
    originally = state["original_query"]
    final_prompt = f"""
    Eres un asistente turístico especializado en la Carretera Austral (Patagonia, Chile) operando en un sistema RAG.

    Políticas de respuesta (cumple TODAS):
    1) Usa EXCLUSIVAMENTE la información del contexto proporcionado.
    2) Si la información solicitada no aparece en el contexto, responde explícitamente: "Esta información no está disponible en los documentos analizados" y solo si la consulta está relacionada con la carretera austral, sugiere qué datos faltan buscar (p. ej., horarios, temporada, accesos).
    3) Cita las fuentes cuando sea posible usando metadata disponible (página, título, fuente, sección).
    4) Prioriza utilidad práctica: tramos, tiempos/ distancias aproximadas, atractivos, accesos, estado de la ruta, clima/temporada, ferries (empalmes, requisitos, reservas), permisos (CONAF), combustible.
    5) No inventes ni extrapoles datos; no supongas horarios ni precios si no están en el contexto.
    6) Estructura la salida de forma clara: Resumen breve de la respuesta.
    7) Regla especial de consistencia geográfica y temporal: Si el contexto presenta datos de diferentes lugares o fechas, indica explícitamente a qué ubicación o periodo pertenece cada dato. No combines información de distintos periodos o lugares como si fueran del mismo.

    Ten en cuenta las siguientes notas:
    - PN es sinónimo de Parque Nacional.
    - P.N. es sinónimo de Parque Nacional.
    - MN es sinónimo de Monumento Natural.
    - M.N. es sinónimo de Monumento Natural.

    Contexto:
    {context}

    Pregunta:
    {originally}

    Respuesta:
    """
    try:
        with get_openai_callback() as cb:
            resp = llm.invoke(final_prompt)
            answer = resp.content
            steps = state["workflow_steps"]
            steps.append(f"generate: {len(answer)} chars, {cb.total_tokens} tokens")
            return {"answer": answer, "workflow_steps": steps, "total_tokens": cb.total_tokens}
    except Exception as e:
        return {"answer": f"Error generando respuesta: {e}", "workflow_steps": state["workflow_steps"]}

# Router

def decide_next(state: TourismRAGState) -> str:
    print("  🤔 Estamos mejorando la consulta...")
    if state["should_rewrite"] and state["retrieval_attempts"] <= 2:
        return "rewrite"
    return "generate"

# Construcción del grafo
workflow = StateGraph(TourismRAGState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade", grade_node)
workflow.add_node("rewrite", rewrite_node)
workflow.add_node("generate", generate_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade")
workflow.add_conditional_edges("grade", decide_next, {"rewrite": "rewrite", "generate": "generate"})
workflow.add_edge("rewrite", "retrieve")
workflow.add_edge("generate", END)

rag_graph = workflow.compile()
print("✅ Grafo Agentic compilado")


✅ Grafo Agentic compilado


In [5]:
# 🚀 Ejecución: función helper para turismo Carretera Austral

def run_agentic_tourism_rag(query: str, verbose: bool = True) -> Dict[str, Any]:
    initial: TourismRAGState = {
        "query": query,
        "original_query": query,
        "documents": [],
        "sources": [],
        "doc_quality_score": 0.0,
        "retrieval_attempts": 0,
        "should_rewrite": False,
        "answer": "",
        "workflow_steps": [],
        "total_tokens": 0,
    }
    if verbose:
        print("\n🚀 RAG AGENTIC para turismo en la Carretera Austral")
        print(f"❓ {query}")
        print("="*60)
    t0 = time.time()
    result = rag_graph.invoke(initial)
    result["execution_time"] = time.time() - t0
    print("\n🤖 RESPUESTA:\n" + "-"*40)
    print(result.get("answer", ""))

    if verbose:
        print("\n🔍 WORKFLOW:")
        for i, s in enumerate(result.get("workflow_steps", []), 1):
            print(f"  {i}. {s}")
    return result

print("✅ Listo. Usa run_agentic_tourism_rag('tu pregunta')")


✅ Listo. Usa run_agentic_tourism_rag('tu pregunta')


In [12]:
run_agentic_tourism_rag('donde están las termas del amarillo?', verbose=False)

  🔍 Recuperando documentos para: donde están las termas del amarillo?
  🤔 Estamos mejorando la consulta...
  🔍 Recuperando documentos para: Ubicación de las Termas del Amarillo (o Termas El Amarillo): coordenadas y dónde están; ¿en qué tramo o localidad, específicamente en la Carretera Austral / Ruta 7? cómo llegar desde la población más cercana, accesos/entrada, estado del camino y necesidad de 4x4, distancia y tiempo de viaje, opciones de transporte (auto/4x4, moto, bus, ferry/barcaza), horarios/itinerario, restricciones (clima, cierres, permisos) y logística (combustible, estacionamiento, contacto)
  🤔 Estamos mejorando la consulta...

🤖 RESPUESTA:
----------------------------------------
Resumen: Están a 25 km al sur de Chaitén por la Ruta 7 (Carretera Austral), con acceso pavimentado en ese tramo.

Detalles prácticos:
- Tramo: Chaitén – La Junta (Aysén Patagonia Queulat).
- Acceso: Ruta 7 hacia el sur desde Chaitén; los primeros 25 km son pavimentados hasta las Termas del Amarillo

{'query': 'Ubicación de las Termas del Amarillo (o Termas El Amarillo): coordenadas y dónde están; ¿en qué tramo o localidad, específicamente en la Carretera Austral / Ruta 7? cómo llegar desde la población más cercana, accesos/entrada, estado del camino y necesidad de 4x4, distancia y tiempo de viaje, opciones de transporte (auto/4x4, moto, bus, ferry/barcaza), horarios/itinerario, restricciones (clima, cierres, permisos) y logística (combustible, estacionamiento, contacto)',
 'original_query': 'donde están las termas del amarillo?',
 'documents': [Document(metadata={'page': 19, 'source': '/Users/pablolastrabachmann/DiploGenAI/rag_langgraph/guia_carretera_1.pdf', 'block_id': 0, 'source_title': 'Guía Carretera Austral', 'collection': 'carretera_austral', 'chunk_id': 28, 'categoria': 'turismo_carretera_austral', 'char_count': 1023, '_id': '32662150-1793-478c-9990-95ed3658b980', '_collection_name': 'carretera_austral'}, page_content='P. 19\nTRAMO | 2\nGuía de viajes Carretera Austral Ays

In [7]:
from datasets import Dataset

eval_questions = [
    "¿Dónde comienza la carretera Austral?",
    "¿Con que otro nombre se le conoce a la junta?"
    # ...
]

# Opcional: ground truths cortas (1–3 oraciones) por pregunta
ground_truths = [
    "La Carretera Austral comienza en la ciudad de Puerto Montt, Región de Los Lagos",
    "La junta es conocida también como El Pueblo del Encuentro"
    # ...
]

answers, contexts_list = [], []
for q in eval_questions:
    r = run_agentic_tourism_rag(q, verbose=False)
    answers.append(r.get("answer", ""))
    contexts_list.append([d.page_content for d in r.get("documents", [])])

data = {"question": eval_questions, "answer": answers, "contexts": contexts_list}
if len(ground_truths) == len(eval_questions):
    data["ground_truths"] = ground_truths

dataset = Dataset.from_dict(data)

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
# Si tienes ground truths y quieres incluir correctness:
from ragas.metrics import answer_correctness

metrics = [faithfulness, answer_relevancy, context_precision, context_recall]
metrics.append(answer_correctness)  # si tienes ground_truths

# Wrappers compatibles con ragas 0.3.x: usar los Wrapper con firmas correctas
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings as LCOpenAIEmbeddings

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))  # evaluador económico
ragas_emb = LangchainEmbeddingsWrapper(LCOpenAIEmbeddings(model="text-embedding-3-large"))

results = evaluate(
    dataset=dataset,
    metrics=metrics,
    llm=ragas_llm,
    embeddings=ragas_emb,
    column_map={
        "question": "question",
        "answer": "answer",
        "contexts": "contexts",
        "reference": "ground_truths",
    },
)

df = results.to_pandas()
print(df)            # métricas por pregunta
print(results)       # promedios globales

  🔍 Recuperando documentos para: ¿Dónde comienza la carretera Austral?
  🤔 Estamos mejorando la consulta...

🤖 RESPUESTA:
----------------------------------------
Resumen breve: La Carretera Austral comienza en la ciudad de Puerto Montt, específicamente en la Plaza de Armas, Región de Los Lagos.

Fuentes:
- Guía de viajes Carretera Austral Aysén Patagonia, Tramo 1, p. 17: “La Carretera Austral comienza en la ciudad de Puerto Montt, Región de Los Lagos.”
- Guía de viajes Carretera Austral Aysén Patagonia, sección “Inicio de la Carretera Austral”: “...desde su inicio en la Plaza de Armas de Puerto Montt...”
  🔍 Recuperando documentos para: ¿Con que otro nombre se le conoce a la junta?
  🤔 Estamos mejorando la consulta...
  🔍 Recuperando documentos para: Buscar nombres alternativos de "La Junta" (localidad/pueblo/caserío/estación/cruce) en la Carretera Austral/Ruta 7: otros nombres oficiales, históricos, toponímicos, apodos, variantes ortográficas y traducciones (La Junta, la junta, junta

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                                      user_input  \
0          ¿Dónde comienza la carretera Austral?   
1  ¿Con que otro nombre se le conoce a la junta?   

                                  retrieved_contexts  \
0  [Carretera Austral: Viaje en auto hasta parque...   
1  [P. 19\nTRAMO | 2\nGuía de viajes Carretera Au...   

                                            response  \
0  Resumen breve: La Carretera Austral comienza e...   
1  Resumen breve de la respuesta:\nLa Junta tambi...   

                                           reference  faithfulness  \
0  La Carretera Austral comienza en la ciudad de ...           1.0   
1  La junta es conocida también como El Pueblo de...           1.0   

   answer_relevancy  context_precision  context_recall  answer_correctness  
0          0.878861                1.0             1.0            0.633211  
1          0.568811                1.0             1.0            0.470928  
{'faithfulness': 1.0000, 'answer_relevancy': 0.7238, 'context_p